In [1]:
from aitk.robots.cart import CartWorld

In [2]:
world = CartWorld()

Random seed set to: 6775859


In [3]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [4]:
robot = world.robots[0]
robot.watch(show_robot=False)

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

In [5]:
import random
import math
# 12 degrees:
theta_max = 12 * math.pi / 180
print("theta_max is", theta_max)

def balance(robot):
    robot.reward += 1
    x, x_v, theta, theta_v = robot.get_observation()
    ok = ((-2.4 < x < 2.4) and
          (-theta_max < theta < theta_max) and
          (robot.reward < 200))
    if ok:
        if theta > 0:
            robot.move(1)
        else:
            robot.move(-1)
    else:
        return robot.reward # out of bounds

theta_max is 0.20943951023931953


In [10]:
world.reset()
reward = world.run([balance], real_time=False)
print("Reward:", reward)

Using random seed: 6775859


0it [00:00, ?it/s]

Simulation stopped at: 00:00:0.66; speed 11.08 x real time
Reward: 34
